<a href="https://colab.research.google.com/github/FabioSicca/Organizacion-de-datos/blob/main/Spark1_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

34)¿Quién es el usuario que más ha bloqueado a otros? (⭐)

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=56093717e4d23a879a0d529bb3f4b6f6777d1ea531e1999804409009bb3cddec
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
type(sc)

pyspark.context.SparkContext

In [ ]:
logs = spark.read.csv("/content/drive/MyDrive/Wikipedia dataset/logs.csv", header=True, inferSchema=True, multiLine=True, escape='"')

In [ ]:
logs.show(10)

+-------+-------------------+--------------------+--------------+--------------+--------------------+-------+------+--------------------+
|item_id|          timestamp|contributor_username|contributor_id|contributor_ip|             comment|logtype|action|               title|
+-------+-------------------+--------------------+--------------+--------------+--------------------+-------+------+--------------------+
|      1|2004-12-23 07:10:09|              Sabbut|          2075|          null|el contenido era:...| delete|delete|    Verificacionismo|
|      2|2004-12-23 07:14:09|              Sabbut|          2075|          null|el contenido era:...| delete|delete|            Rapunzel|
|      3|2004-12-23 07:26:10|              Sabbut|          2075|          null|el contenido era:...| delete|delete|    Macromedia Flash|
|      4|2004-12-23 07:55:09|              Sanbec|          7856|          null|Sin información d...| delete|delete|Archivo:Gnu head.png|
|      5|2004-12-23 08:02:22|     

In [ ]:
rdd = logs.rdd

Filtro el rdd por aquellos que tienen como acción "block", es decir me quedo con todas las acciones que hayan sido bloquear

In [ ]:
rdd = rdd.filter(lambda x: x.action == "block")

Genero registros de tipo (clave, valor) = (contributor_id, contador) con el map para poder usar reduceByKey con el cual sumo la cantidad de elementos para cada clave


In [ ]:
rdd2 = rdd.map(lambda x: (x.contributor_username, 1)).reduceByKey(lambda f1,f2: f1 + f2)
rdd2.take(5)

[('Dodo', 599),
 ('Comae', 21),
 ('Sanbec', 526),
 ('Lourdes Cardenal', 2357),
 ('Sabbut', 1041)]

In [ ]:
usuario = rdd2.reduce(lambda f1,f2: f1 if (f1[1] > f2[1]) else f2)
print(usuario)

('Magister Mathematicae', 29233)


El usuario que más ha bloqueado es 'Magister Mathematicae' con 29233 bloqueos

#Código en limpio:

In [ ]:
rdd = logs.rdd
rdd = rdd.filter(lambda x: x.action == "block")
rdd2 = rdd.map(lambda x: (x.contributor_username, 1)).reduceByKey(lambda f1,f2: f1 + f2)
rdd2.reduce(lambda f1,f2: f1 if (f1[1] > f2[1]) else f2)

('Magister Mathematicae', 29233)